In [1]:
import cv2
from PIL import Image, ImageFilter
import fitz
import os
import pandas as pd
import numpy as np
import shutil
import torch
import warnings
warnings.filterwarnings('ignore')
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"Tesseract-OCR\\tesseract.exe"
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# trained yolov5 model to extract individual cell from the cells group image
print('loading block extractor........')
yolo_block = {'path':'block.pt', 'force_reload':True}
model_block = torch.hub.load('ultralytics/yolov5', 'custom', **yolo_block) 

# trained yolov5 model to extract features from the cell
print('loading feature extractor')
yolo_feature = {'path':'feature.pt', 'force_reload':True}
model_feature = torch.hub.load('ultralytics/yolov5', 'custom', **yolo_feature) 

loading block extractor........


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\spsoft/.cache\torch\hub\master.zip
YOLOv5  2023-5-3 Python-3.9.13 torch-1.13.0+cpu CPU

Fusing layers... 


requirements: C:\Users\spsoft\.cache\torch\hub\requirements.txt not found, check failed.


Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\spsoft/.cache\torch\hub\master.zip


loading feature extractor


YOLOv5  2023-5-3 Python-3.9.13 torch-1.13.0+cpu CPU

Fusing layers... 


requirements: C:\Users\spsoft\.cache\torch\hub\requirements.txt not found, check failed.


Model summary: 157 layers, 7029004 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
# list_values = ['temp/pdf_1.png', 'temp/pdf_3.png', 'temp/pdf_8.png', 'temp/pdf_4.png']
# order = {'temp/pdf_3.png': 1, 'temp/pdf_1.png': 0, 'temp/pdf_8.png': 3, 'temp/pdf_4.png': 2}

# sorted_list = sorted(list_values, key=lambda x: order[x])
# print(sorted_list)

In [9]:
#converting pdf to image using PyMuPDF
def pdf_image(path, image_path):
    '''parameter path contains single_pdf_file location'''
    dir_name, file_name = os.path.split(path)
    img_name = os.path.splitext(file_name)[0]
    zoom_x = 2.0  # horizontal zoom
    zoom_y = 2.0  # vertical zoom
    mat = fitz.Matrix(zoom_x, zoom_y)
    #patht="/code/./coreservice/pdf"
    doc = fitz.open(path)
    #os.chdir(patht)
#     image_path = 'D:/poll/18022023/temp_image'
    image_pages=[]
    order = {}
    for page in doc:  # iterate through the pages
        pix = page.get_pixmap(matrix=mat)  # render page to an image
        photo = "{}_{}.png".format(img_name, page.number)
        photo_path = os.path.join(image_path, photo).replace('\\', '/')
        order[photo_path] = page.number
        pix.save(photo_path)
        image_pages.append(photo_path)
        print('image name:', photo_path)
    
    image_lst = sorted(image_pages, key = lambda x:order[x])
    return image_lst

In [15]:
def text_process(text):
    text = text.replace('\u200c','')
    text = text.replace("'", "")
    text = text.replace("\n", "")
    text = text.replace('౦','')
    text = text.replace('స్తీ', 'స్త్రీ')
    return text

In [ ]:
files_path = r'test'
img_path = r'temp_image'
result_path = r'extracted_result'
final_data = []
block_dataframes = []
feature_dataframes = []
for direct, _, filenames in os.walk(files_path):
    writer = pd.ExcelWriter(f'{result_path}\\extracted_data.xlsx', engine='xlsxwriter')
    for filename in filenames:
        name_file, ext = os.path.splitext(filename)
        print('name:', name_file)
        
        if ext.lower() == '.pdf':
            sheet_pdf = []         
            pdf_path = os.path.join(direct, filename).replace('\\', '/')
            print('pdf to image conversion started.......')
            image_data = pdf_image(pdf_path, img_path)
            print('............... image conversion completed')
            if len(image_data) < 4:
                print('no image data')
                [os.remove(i) for i in image_data]
                no_pages = pd.DataFrame([{'instruction':'Please check the given file it has to be atleast 4 pages'}])
                no_pages.to_excel(f'{result_path}\\{name_file}.xlsx', index = False)
                continue
            else:
                rmv_lst = image_data[:2] + [image_data[-1]]
                [os.remove(i) for i in rmv_lst]
                image_data = image_data[2:-1]         
            img_cnt = 0
            for image in image_data:                
                img_cnt += 1
                # block extractor
                img = cv2.imread(image)
                print('blocks extraction started.............')
                blocks_image = model_block(img)
                df = blocks_image.pandas().xyxy[0]
#                 df = df.applymap(lambda x: int(x) if isinstance(x, float) else x)
                df[['xmin', 'ymin', 'xmax', 'ymax']] = df[['xmin', 'ymin', 'xmax', 'ymax']].astype(int)
                block_dataframes.append(df)
                print('feature extraction started.......')
                main_dir_name = os.path.splitext(os.path.split(image)[-1])[0]
                main_dir = f'test_crop/{main_dir_name}_{img_cnt}'
                os.mkdir(main_dir)
                shutil.copy(image, main_dir)
                cnt = 0
                for xmin,ymin,xmax,ymax,thresh, annot_b in zip(df["xmin"],df["ymin"],df["xmax"],df["ymax"], df['confidence'], df['name']):
                    cnt += 1
                    block_image = img[ymin:ymax, xmin:xmax]
                    try:
                        sub_dir = f'{main_dir}/{annot_b}_{cnt}'
                        os.mkdir(sub_dir)
                        cv2.imwrite(f'{sub_dir}/{annot_b}.png', block_image)
                    except:
                        pass
                    features_image = model_feature(block_image)
                    feat_df = features_image.pandas().xyxy[0].sort_values(by = 'confidence', ascending = False)
                    feat_df = feat_df.drop_duplicates(subset = 'name', keep = 'first')
#                     feat_df = features_image.pandas().xyxy[0].applymap(lambda x: int(x) if isinstance(x, float) else x)
                    feat_df[['xmin', 'ymin', 'xmax', 'ymax']] = feat_df[['xmin', 'ymin', 'xmax', 'ymax']].astype(int)
                    feature_dataframes.append(feat_df)
                    temp_data = {}
                    for xmin_f, ymin_f, xmax_f,\
                    ymax_f,thresh_f, annot_f in zip(feat_df['xmin'], feat_df['ymin'],\
                                               feat_df['xmax'], feat_df['ymax'],feat_df['confidence'], feat_df['name']):
                        feat_img = block_image[ymin_f:ymax_f, xmin_f:xmax_f]
#                         print('feat image stored at:', f'D:/poll/pdf_data/test_crop/{name}_{cnt}/{name}.png')
                        cv2.imwrite(f'{sub_dir}/{annot_f}.png', feat_img)
                        temp_data[annot_f] = text_process(pytesseract.image_to_string(feat_img, lang = 'eng', config = '--psm 6'))
#                         if annot_f == 'id':
#                 ######################################################################
#                             temp_data['Photo'] = Image.fromarray(block_image[ymax_f:, xmin_f:xmax_f])
                    final_data.append(temp_data)
                    sheet_pdf.append(temp_data)
                    print(temp_data)
                    print('-----------------------------------------------------')
                os.remove(image)
            print('----------done-------------')
            result_pdf = pd.DataFrame(sheet_pdf)
#             result_pdf = result_pdf[['sno','id','Name','Gender','Age','Gaurdian','HouseNo', 'Photo']]
#             result_pdf.columns = ['Serial Number', 'Registration Number', 'Name',
#                                  'Gender', 'Age', 'Guardian', 'House Number', 'Photo']
            result_pdf = result_pdf[['sno','id','Name','Gender','Age','Gaurdian','HouseNo']]
            result_pdf.columns = ['Serial Number', 'Registration Number', 'Name',
                                 'Gender', 'Age', 'Guardian', 'House Number']
            result_pdf.to_excel(writer, sheet_name = name_file, index = False)
    writer.close()

             
                

name: Popuppage(1)
pdf to image conversion started.......
image name: temp_image/Popuppage(1)_0.png
image name: temp_image/Popuppage(1)_1.png
image name: temp_image/Popuppage(1)_2.png
image name: temp_image/Popuppage(1)_3.png
image name: temp_image/Popuppage(1)_4.png
image name: temp_image/Popuppage(1)_5.png
image name: temp_image/Popuppage(1)_6.png
image name: temp_image/Popuppage(1)_7.png
image name: temp_image/Popuppage(1)_8.png
image name: temp_image/Popuppage(1)_9.png
image name: temp_image/Popuppage(1)_10.png
image name: temp_image/Popuppage(1)_11.png
image name: temp_image/Popuppage(1)_12.png
image name: temp_image/Popuppage(1)_13.png
image name: temp_image/Popuppage(1)_14.png
image name: temp_image/Popuppage(1)_15.png
image name: temp_image/Popuppage(1)_16.png
image name: temp_image/Popuppage(1)_17.png
image name: temp_image/Popuppage(1)_18.png
image name: temp_image/Popuppage(1)_19.png
image name: temp_image/Popuppage(1)_20.png
image name: temp_image/Popuppage(1)_21.png
image 

{'id': 'URH0891532', 'sno': '34'}
-----------------------------------------------------
{'id': 'URH2130060', 'sno': '31', 'Name': 'Kanukaiah Marka'}
-----------------------------------------------------
{'id': 'URH0865404', 'sno': '43', 'Name': 'Punnam Perumandia'}
-----------------------------------------------------
{'id': 'URH2130524', 'sno': '56', 'HouseNo': 'FEMALE', 'Name': '-Sammakka Merugu'}
-----------------------------------------------------
{'sno': '47', 'id': 'DVD1340645', 'Gaurdian': 'Saraiah', 'Name': 'Ravi Gurram', 'Age': '40'}
-----------------------------------------------------
{'id': 'DVD1340926', 'sno': '37', 'Name': 'Nirmala Rathnam:', 'Age': '7'}
-----------------------------------------------------
{'id': 'URH2131241', 'sno': '52', 'Name': 'Thirupathi Bonagani'}
-----------------------------------------------------
{'id': 'URH2127835', 'sno': '38', 'Name': '‘Saraiah Gurram', 'Gaurdian': 'Gattayya'}
-----------------------------------------------------
{'id': 'UR

{'id': 'DVD1341452', 'sno': '116', 'Gaurdian': 'Sammaiah', 'Name': '- Thirupathi Chirra'}
-----------------------------------------------------
{'id': 'URH2129401', 'sno': '113', 'Name': '‘Sulochana Inige', 'Age': '55'}
-----------------------------------------------------
{'id': 'URH0431346', 'sno': '95', 'Name': 'Ramesh Maddela', 'Age': '36'}
-----------------------------------------------------
{'id': 'URHO786600', 'sno': '96°', 'Age': '30', 'Name': 'Swarnalatha Maddela'}
-----------------------------------------------------
{'id': 'URH2127595', 'sno': '106', 'Name': 'Ravi Bomma', 'Age': '55'}
-----------------------------------------------------
{'id': 'URH1303171', 'sno': 'o1', 'Name': 'Manasa Avula', 'Age': '24', 'Gaurdian': 'Devendar'}
-----------------------------------------------------
{'id': 'URH2128643.', 'sno': '112', 'Name': 'Mohan Rao Inige', 'Gaurdian': 'Kanakaiah', 'Age': '60'}
-----------------------------------------------------
{'id': 'URH2128189', 'sno': '92', 'Nam

{'id': 'URH0891334', 'sno': '173', 'Age': '28'}
-----------------------------------------------------
{'id': 'URH2129542', 'sno': '155', 'Name': 'Raajaiah Panjala', 'Age': '83'}
-----------------------------------------------------
{'id': 'URH1248491', 'sno': '164', 'Age': '25'}
-----------------------------------------------------
{'id': 'URH1301589', 'sno': '167', 'Name': 'Latha Chenu', 'Gaurdian': 'Balakrushna.'}
-----------------------------------------------------
{'sno': 'E180', 'id': 'DVD1341056'}
-----------------------------------------------------
{'id': 'URH2130177,', 'sno': '176'}
-----------------------------------------------------
{'id': 'URH2129369', 'sno': '151', 'Name': 'Parmila Panjala_', 'Age': '45'}
-----------------------------------------------------
{'id': 'URH0430611', 'sno': '4179', 'Name': '‘Srinivas Avula_', 'Age': '48', 'Gaurdian': 'Sailu Avula'}
-----------------------------------------------------
{'id': 'URH2130243', 'sno': '172', 'Name': '‘Lakshmi Chenu

{'id': 'URH2129484', 'sno': '235', 'Name': 'Lacchamma Aavula', 'HouseNo': '“FEMALE.'}
-----------------------------------------------------
{'id': 'URH2130227,', 'sno': '224', 'Name': 'Mallakka Edula_', 'Age': '50', 'HouseNo': 'FEMALE'}
-----------------------------------------------------
{'id': 'URH0865735,', 'sno': '211', 'Gaurdian': 'Santosh Salla', 'Name': 'Ramadevi Salla'}
-----------------------------------------------------
{'id': 'URH0865792', 'sno': '226', 'Gaurdian': 'Sampath Avula', 'Name': 'Sumalatha Avula'}
-----------------------------------------------------
{'id': 'URH1012772', 'sno': '215', 'Name': 'Neeraja Gandala', 'Gaurdian': 'Thirupathi Gandala', 'HouseNo': 'FEMALE'}
-----------------------------------------------------
{'id': 'URH2130847,', 'sno': '236', 'Name': '-Papaaiah Aavula'}
-----------------------------------------------------
{'id': 'URH2130425', 'sno': '233'}
-----------------------------------------------------
{'id': 'URH2128064', 'sno': '£227'}
-----